In [1]:
import pandas as pd
import psycopg2

In [2]:
hv_map = pd.read_csv('discrete_prio_fields.csv')

In [3]:
db_connection = psycopg2.connect(user='postgres',
                                 password='220189BScAthens',
                                 host='localhost',
                                 port='5432',
                                 database='UK_BB')

In [19]:
mapping_type = hv_map['mappingType'].unique()
# mapping_type.shape
ctype_1 = mapping_type[0]
ctype_2 = mapping_type[1]

(2,)

In [5]:
query_concept_1 = "select concept_id_2 \
                    from vocab.concept join vocab.concept_relationship on concept_id = concept_id_1 \
                    where vocabulary_id = 'UK Biobank' and concept_code = '%(code)s' \
                    and relationship_id = 'Maps to'"
query_concept_2 = "select concept_id_2 \
                    from vocab.concept join vocab.concept_relationship on concept_id = concept_id_1 \
                    where vocabulary_id = 'UK Biobank' and concept_code = '%(code)s' \
                    and relationship_id = 'Maps to unit'"

In [17]:
codes = hv_map['sourceCode'].unique().tolist()

AttributeError: 'DataFrameGroupBy' object has no attribute 'unique'

In [7]:
cursor = db_connection.cursor()

all_possible = []
for c in codes:
    field_id = c
    field_name = hv_map[hv_map['sourceCode']==c]['sourceName'].unique()[0]
    cursor.execute(query_concept_1, {'code': c})
    a8_concept_1 = cursor.fetchone()
    cursor.execute(query_concept_2, {'code': c})
    a8_concept_2 = cursor.fetchone()
    hv_concept_1 = hv_map[(hv_map['sourceCode']==c) & (hv_map['mappingType']==ctype_1)]['conceptId'].tolist()
    hv_concept_2 = hv_map[(hv_map['sourceCode']==c) & (hv_map['mappingType']==ctype_2)]['conceptId'].tolist()
    if a8_concept_1 is None:
        a8_concept_1 = (0,)
    if a8_concept_2 is None:
        a8_concept_2 = (0,)
    if not hv_concept_1:
        hv_concept_1 = [0]
    if not hv_concept_2:
        hv_concept_2 = [0]
    all_possible.append([field_id, field_name, a8_concept_1[0], a8_concept_2[0], hv_concept_1[0], hv_concept_2[0]])
print(all_possible)

[[30746, 'Glucose reportability', 35814468, 0, 4040635, 4282459], [1558, 'Alcohol intake frequency.', 35810226, 0, 4052351, 45885326], [30885, 'Urate missing reason', 35814563, 0, 44803923, 4229128], [30785, 'LDL direct missing reason', 35814494, 0, 44803923, 4329038], [30846, 'Total bilirubin reportability', 35814536, 0, 4040635, 4196330], [30652, 'Aspartate aminotransferase aliquot', 35814401, 0, 4227695, 4094657], [30813, 'Phosphate correction level', 35814512, 0, 0, 4125861], [30254, 'Reticulocyte count acquisition route', 35814310, 0, 4263581, 36307628], [2453, 'Cancer diagnosed by doctor', 35810298, 0, 0, 443392], [30264, 'Mean reticulocyte volume acquisition route', 35814315, 0, 3049858, 42531073], [30786, 'LDL direct reportability', 35814495, 0, 4040635, 4329038], [30753, 'Glycated haemoglobin (HbA1c) correction level', 35814471, 0, 0, 37118746], [30653, 'Aspartate aminotransferase correction level', 35814402, 0, 0, 4094657], [30814, 'Phosphate correction reason', 35814513, 0, 

In [8]:
df_results = pd.DataFrame(all_possible)
df_results.columns = ['field_id', 'field_name', f'a8_{ctype_1}', f'a8_{ctype_2}', f'hv_{ctype_1}', f'hv_{ctype_2}']
df_results[f'match_{ctype_1}'] = (df_results[f'a8_{ctype_1}'] == df_results[f'hv_{ctype_1}'])
df_results[f'match_{ctype_2}'] = (df_results[f'a8_{ctype_2}'] == df_results[f'hv_{ctype_2}'])


In [9]:
df_results

,field_id,field_name,a8_EVENT,a8_VALUE,hv_EVENT,hv_VALUE,match_EVENT,match_VALUE
0,30746,Glucose reportability,35814468,0,4040635,4282459,False,False
1,1558,Alcohol intake frequency.,35810226,0,4052351,45885326,False,False
2,30885,Urate missing reason,35814563,0,44803923,4229128,False,False
3,30785,LDL direct missing reason,35814494,0,44803923,4329038,False,False
4,30846,Total bilirubin reportability,35814536,0,4040635,4196330,False,False
...,...,...,...,...,...,...,...,...
201,30634,Apolipoprotein A correction reason,35814389,0,4077846,4337512,False,False
202,30723,Cystatin C correction level,35814451,0,0,4136584,False,False
203,30695,Cholesterol missing reason,35814432,0,44803923,4224820,False,False
204,30635,Apolipoprotein A missing reason,35814390,0,44803923,4337512,False,False


In [10]:
query_details = "select domain_id, vocabulary_id from vocab.concept where concept_id = '%(code2)s'"

In [11]:
a8_details_1 = []
a8_details_2 = []
hv_details_1 = []
hv_details_2 = []
n = df_results.shape[0]
for i in range(n):
    cursor.execute(query_details, {'code2': df_results[f'a8_{ctype_1}'].tolist()[i]})
    a8_c1 = cursor.fetchone()
    a8_details_1.append(a8_c1)
    cursor.execute(query_details, {'code2': df_results[f'a8_{ctype_2}'].tolist()[i]})
    a8_c2 = cursor.fetchone()
    a8_details_2.append(a8_c2)
    cursor.execute(query_details, {'code2': df_results[f'hv_{ctype_1}'].tolist()[i]})
    hv_c1 = cursor.fetchone()
    hv_details_1.append(hv_c1)
    cursor.execute(query_details, {'code2': df_results[f'hv_{ctype_2}'].tolist()[i]})
    hv_c2 = cursor.fetchone()
    hv_details_2.append(hv_c2)

In [12]:
df_results[f'a8_{ctype_1}_details'] = a8_details_1
df_results[f'a8_{ctype_2}_details'] = a8_details_2
df_results[f'hv_{ctype_1}_details'] = hv_details_1
df_results[f'hv_{ctype_2}_details'] = hv_details_2

In [16]:
pd.set_option('display.max_rows', 500)
df_results[df_results['match_EVENT'] == False] #.count()

,field_id,field_name,a8_EVENT,a8_VALUE,hv_EVENT,hv_VALUE,match_EVENT,match_VALUE,a8_EVENT_details,a8_VALUE_details,hv_EVENT_details,hv_VALUE_details
0,30746,Glucose reportability,35814468,0,4040635,4282459,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Observation, SNOMED)","(Observation, SNOMED)"
1,1558,Alcohol intake frequency.,35810226,0,4052351,45885326,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Observation, SNOMED)","(Meas Value, LOINC)"
2,30885,Urate missing reason,35814563,0,44803923,4229128,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Observation, SNOMED)","(Observation, SNOMED)"
3,30785,LDL direct missing reason,35814494,0,44803923,4329038,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Observation, SNOMED)","(Observation, SNOMED)"
4,30846,Total bilirubin reportability,35814536,0,4040635,4196330,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Observation, SNOMED)","(Observation, SNOMED)"
5,30652,Aspartate aminotransferase aliquot,35814401,0,4227695,4094657,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Procedure, SNOMED)","(Observation, SNOMED)"
6,30813,Phosphate correction level,35814512,0,0,4125861,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Metadata, None)","(Observation, SNOMED)"
7,30254,Reticulocyte count acquisition route,35814310,0,4263581,36307628,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Measurement, SNOMED)","(Meas Value, LOINC)"
8,2453,Cancer diagnosed by doctor,35810298,0,0,443392,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Metadata, None)","(Condition, SNOMED)"
9,30264,Mean reticulocyte volume acquisition route,35814315,0,3049858,42531073,False,False,"(Observation, UK Biobank)","(Metadata, None)","(Measurement, LOINC)","(Meas Value, LOINC)"


In [ ]:
df_results.to_csv('all_concepts_discrete_prio.csv', sep=',', index=False)